## Import Dependencies

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Data Collection and Preprocessing

In [2]:
df = pd.read_csv("movies.csv")
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
df.shape

(4803, 24)

In [4]:
df.isnull().sum()

,0
index,0
budget,0
genres,28
homepage,3091
id,0
keywords,412
original_language,0
original_title,0
overview,3
popularity,0


In [5]:
selected_features = ["genres", "keywords", "popularity", "tagline", "title", "cast", "director"]
print(selected_features)

['genres', 'keywords', 'popularity', 'tagline', 'title', 'cast', 'director']


In [6]:
for feature in selected_features:
    df[feature] = df[feature].fillna("")

In [7]:
combined_features = df["genres"].astype(str) +" "+ df["keywords"].astype(str) + " " + df["popularity"].astype(str) + " " + df["tagline"].astype(str) + " " + df["title"].astype(str) + " " + df["cast"].astype(str) + " " + df["director"].astype(str)
# The error was due to trying to concatenate string columns with numeric columns.
# To fix this, we convert all the relevant columns to strings using .astype(str) before concatenation.

In [8]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  0.642552 A newlywed couple's h...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      0.857008 A New Yorker in Shanghai Shanghai C...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [9]:
vectorizer = TfidfVectorizer()

In [10]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [11]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 142507 stored elements and shape (4803, 24065)>
  Coords	Values
  (0, 5124)	0.07104187438906007
  (0, 5203)	0.08148674756233969
  (0, 10763)	0.1004748739439352
  (0, 19892)	0.09374170819868682
  (0, 10936)	0.09363250729525044
  (0, 8984)	0.1934882630597334
  (0, 8289)	0.20087062670134076
  (0, 11400)	0.14789003851519936
  (0, 20782)	0.30313341398071625
  (0, 23266)	0.1123510383796569
  (0, 8468)	0.22576000914118463
  (0, 20666)	0.1903932969434866
  (0, 881)	0.25803175522263594
  (0, 2384)	0.25803175522263594
  (0, 10400)	0.20539880433869395
  (0, 21785)	0.04995717206634014
  (0, 23716)	0.11340025738452353
  (0, 17237)	0.0704913896233848
  (0, 17581)	0.23767055042014518
  (0, 6064)	0.25803175522263594
  (0, 19630)	0.13563592596064908
  (0, 23726)	0.21384946786222409
  (0, 24029)	0.18157772178077283
  (0, 19599)	0.19694814081516385
  (0, 20423)	0.18628778860109438
  :	:
  (4801, 4354)	0.25999006249469625
  (4801, 23880)	0.2599

#getting similarity score by cosine similarity

In [12]:
similarity = cosine_similarity(feature_vectors)

In [13]:
print(similarity)

[[1.         0.06502598 0.03071383 ... 0.         0.         0.        ]
 [0.06502598 1.         0.02496525 ... 0.04092152 0.         0.        ]
 [0.03071383 0.02496525 1.         ... 0.         0.04179736 0.        ]
 ...
 [0.         0.04092152 0.         ... 1.         0.         0.05194031]
 [0.         0.         0.04179736 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.05194031 0.         1.        ]]


In [14]:
similarity.shape

(4803, 4803)

### Getting movie name from user

In [37]:
movie_name = input("Enter your favourite movie name: ")

Enter your favourite movie name: Avengers


In [38]:
#creating list with all the movie list in the dataset
list_of_all_titles = df["title"].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [39]:
#Finding the close match for movie name for input movie_name
close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(close_match)

['The Avengers', 'Swingers']


In [40]:
most_close_match = close_match[0]
print(most_close_match)

The Avengers


In [41]:
#finding the index of movie with title
index_of_movie = df[df["title"] == most_close_match]["index"].values[0]
print(index_of_movie)

16


In [42]:
# getting list of similar movies
similarity_score = list(enumerate(similarity[index_of_movie]))
print(similarity_score)

[(0, np.float64(0.030441122692849362)), (1, np.float64(0.022341735314309167)), (2, np.float64(0.0314857253099605)), (3, np.float64(0.010094780137314956)), (4, np.float64(0.04833846367533129)), (5, np.float64(0.014118791096884655)), (6, np.float64(0.019266639764601426)), (7, np.float64(0.6508613230273151)), (8, np.float64(0.008529252854397974)), (9, np.float64(0.12192106254439469)), (10, np.float64(0.0634402571165974)), (11, np.float64(0.00993781013722263)), (12, np.float64(0.012896997642299157)), (13, np.float64(0.014329045261904158)), (14, np.float64(0.15074188019930201)), (15, np.float64(0.03730448801635098)), (16, np.float64(1.0000000000000002)), (17, np.float64(0.020075886899673143)), (18, np.float64(0.023426689470002725)), (19, np.float64(0.019409636723889068)), (20, np.float64(0.09544112277779493)), (21, np.float64(0.022419582405334336)), (22, np.float64(0.010933774468204448)), (23, np.float64(0.03369576269371691)), (24, np.float64(0.015392635814267416)), (25, np.float64(0.008030

In [43]:
len(similarity_score)

4803

In [44]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(16, np.float64(1.0000000000000002)), (7, np.float64(0.6508613230273151)), (85, np.float64(0.337192626651649)), (79, np.float64(0.33498459012607984)), (26, np.float64(0.302148053701683)), (126, np.float64(0.26005963507832586)), (511, np.float64(0.24776795689556905)), (174, np.float64(0.24076369841135206)), (64, np.float64(0.2258932118225053)), (182, np.float64(0.22389644145800633)), (129, np.float64(0.22202054200853974)), (203, np.float64(0.22050069391958316)), (33, np.float64(0.20497942320603849)), (788, np.float64(0.20309087585559873)), (169, np.float64(0.1906725021601097)), (68, np.float64(0.18926916056632023)), (38, np.float64(0.1890474829736578)), (46, np.float64(0.1890106528567661)), (4759, np.float64(0.18772169261612054)), (31, np.float64(0.18320582323585774)), (782, np.float64(0.15801069937420748)), (870, np.float64(0.1561312553928491)), (101, np.float64(0.1524671385193741)), (14, np.float64(0.15074188019930201)), (94, np.float64(0.1333260031985443)), (1294, np.float64(0.12971

In [45]:
#print the name of similar movies based on the index
print("Suggested movies for you: \n")
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df["index"]==index]["title"].values[0]
  if i<30:
    print(i, "", title_from_index)
    i+=1

Suggested movies for you: 

1  The Avengers
2  Avengers: Age of Ultron
3  Captain America: The Winter Soldier
4  Iron Man 2
5  Captain America: Civil War
6  Thor: The Dark World
7  X-Men
8  The Incredible Hulk
9  X-Men: Apocalypse
10  Ant-Man
11  Thor
12  X2
13  X-Men: The Last Stand
14  Deadpool
15  Captain America: The First Avenger
16  Iron Man
17  The Amazing Spider-Man 2
18  X-Men: Days of Future Past
19  The Image Revolution
20  Iron Man 3
21  The Spirit
22  Superman II
23  X-Men: First Class
24  Man of Steel
25  Guardians of the Galaxy
26  Serenity
27  Spawn
28  The Jungle Book
29  What's Your Number?


### Recommendation system

In [48]:
movie_name = input("Enter your favourite movie name: ")
list_of_all_titles = df["title"].tolist()
close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
most_close_match = close_match[0]
index_of_movie = df[df["title"] == most_close_match]["index"].values[0]
similarity_score = list(enumerate(similarity[index_of_movie]))
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print("Suggested movies for you: \n")
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df["index"]==index]["title"].values[0]
  if i<30:
    print(i, "", title_from_index)
    i+=1

Enter your favourite movie name: American Dream
Suggested movies for you: 

1  American Dreamz
2  xXx: State of the Union
3  W.
4  Swing Vote
5  Head of State
6  Chasing Liberty
7  The American President
8  Love Actually
9  Vantage Point
10  American Pie
11  Flags of Our Fathers
12  Music and Lyrics
13  About a Boy
14  Capitalism: A Love Story
15  Far from Heaven
16  O Brother, Where Art Thou?
17  eXistenZ
18  The Conspirator
19  Fahrenheit 9/11
20  Chuck & Buck
21  Role Models
22  The Express
23  Spice World
24  Oliver!
25  Cat People
26  Troy
27  Savage Grace
28  Tangled
29  Men of War


## dumping model

In [49]:
import joblib

In [50]:
joblib.dump(similarity, "similarity.pkl")

['similarity.pkl']

In [51]:
joblib.dump(vectorizer, "vectorizer.pkl")

['vectorizer.pkl']